# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 10.2.5
- **Date :** 17/11/2023
- **Heure :** 14:05
- **Données :** `model_weatherAUS.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** `SMOTE`
- **Conversion :** `numpyfy`
- **Algorithme :** TSF

## Sommaire

1. Initialisation
2. Découpage
3. Rééquilibrage
4. Conversion
5. Modélisation
6. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../data/1df/model_weatherAUS.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,Month,Day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum,NonMesNum
Date,,,,,,,,,,,,,,,,,,,,,
2007-11-01,8.0,24.3,0.0,3.4,6.3,30.0,6.0,20.0,68.0,29.0,...,11,1,15,2.356194,3.926991,2.356194,-35.297591,149.101268,1,2
2007-11-02,14.0,26.9,3.6,4.4,9.7,39.0,4.0,17.0,80.0,36.0,...,11,2,15,0.392699,0.000000,3.141593,-35.297591,149.101268,1,2
2007-11-03,13.7,23.4,3.6,5.8,3.3,85.0,6.0,6.0,82.0,69.0,...,11,3,15,2.356194,1.570796,1.178097,-35.297591,149.101268,1,2
2007-11-04,13.3,15.5,39.8,7.2,9.1,54.0,30.0,24.0,62.0,56.0,...,11,4,15,2.356194,2.748894,3.141593,-35.297591,149.101268,1,2
2007-11-05,7.6,16.1,2.8,5.6,10.6,50.0,20.0,28.0,68.0,49.0,...,11,5,15,5.105088,5.105088,5.890486,-35.297591,149.101268,1,2


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140787 entries, 2007-11-01 to 2017-06-25
Data columns (total 29 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   MinTemp         140787 non-null  float64
 1   MaxTemp         140787 non-null  float64
 2   Rainfall        140787 non-null  float64
 3   Evaporation     140787 non-null  float64
 4   Sunshine        140787 non-null  float64
 5   WindGustSpeed   140787 non-null  float64
 6   WindSpeed9am    140787 non-null  float64
 7   WindSpeed3pm    140787 non-null  float64
 8   Humidity9am     140787 non-null  float64
 9   Humidity3pm     140787 non-null  float64
 10  Pressure9am     140787 non-null  float64
 11  Pressure3pm     140787 non-null  float64
 12  Cloud9am        140787 non-null  float64
 13  Cloud3pm        140787 non-null  float64
 14  Temp9am         140787 non-null  float64
 15  Temp3pm         140787 non-null  float64
 16  RainToday       140787 non-null  int64  
 17  Ra

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Rééquilibrage

In [7]:
# Suréchantillonnage avec SMOTE

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 12)
X_train, y_train = smote.fit_resample(X_train, y_train)

## 4. Conversion

In [8]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [9]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 5. Modélisation

In [10]:
# Instanciation d'un modèle classificateur via un Pipeline

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier

steps = [
    ("concatenate", ColumnConcatenator()), ## Cette étape permet de transformer les jeux de données multivariées en univariées afin qu'ils puissent être traités par `TimeSeriesForestClassifier`.
    ("classify", TimeSeriesForestClassifier()),
]

clf_rf_ts = Pipeline(steps)

In [11]:
# Récupération des paramètres initiaux de la TSF

clf_rf_ts["classify"].get_params()

{'min_interval': 3, 'n_estimators': 200, 'n_jobs': 1, 'random_state': None}

In [12]:
# Entraînement du modèle

clf_rf_ts.fit(X_train, y_train)

Pipeline(steps=[('concatenate', ColumnConcatenator()),
                ('classify', TimeSeriesForestClassifier())])

In [13]:
# Récupération des paramètres ajustés de la TSF

clf_rf_ts["classify"].get_fitted_params()

{'classes': array([0, 1]),
 'intervals': [array([[21, 25],
         [ 3, 21],
         [14, 25],
         [19, 22],
         [ 7, 19]]),
  array([[20, 23],
         [22, 26],
         [19, 24],
         [17, 20],
         [ 7, 10]]),
  array([[16, 19],
         [21, 26],
         [23, 26],
         [ 8, 11],
         [ 8, 16]]),
  array([[13, 25],
         [10, 26],
         [18, 21],
         [20, 23],
         [18, 23]]),
  array([[ 8, 11],
         [ 9, 15],
         [ 7, 16],
         [23, 26],
         [ 6, 19]]),
  array([[19, 26],
         [18, 25],
         [ 4, 11],
         [24, 27],
         [ 9, 20]]),
  array([[22, 25],
         [ 7, 18],
         [21, 26],
         [ 7, 21],
         [ 7, 21]]),
  array([[14, 20],
         [17, 26],
         [24, 27],
         [21, 24],
         [ 7, 16]]),
  array([[24, 27],
         [12, 24],
         [15, 18],
         [19, 26],
         [ 6, 25]]),
  array([[ 0,  5],
         [15, 18],
         [ 0,  7],
         [19, 24],
         [1

In [14]:
# Réalisation des prédictions

y_pred = clf_rf_ts.predict(X_test)

## 6. Évaluation

In [15]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,17458,661
1,3380,1965


In [16]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90     18119
           1       0.75      0.37      0.49      5345

    accuracy                           0.83     23464
   macro avg       0.79      0.67      0.69     23464
weighted avg       0.82      0.83      0.80     23464

